In [1]:
import sklearn
import lightgbm as lgb
import pandas as pd 
from tqdm.autonotebook import tqdm
import numpy as np
import gc
data_dir = './data/'

/usr/local/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)
/usr/local/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
labels = pd.DataFrame()
trains = pd.DataFrame()
tests = pd.DataFrame()
gc.collect()
for i in tqdm(range(1, 6)):
    trains = trains.append(pd.read_csv(data_dir+'train%d.csv'%i, header=None))
    labels = labels.append(pd.read_csv(data_dir+'label%d.csv'%i, header=None))
    tests = tests.append(pd.read_csv(data_dir+'test%i.csv'%i, header=None))
tests = tests.append(pd.read_csv(data_dir+'test6.csv', header=None))

In [3]:
import sklearn.model_selection
def cv(x, y, model, params={}, splits=5):
    clf = model(**params)
    kfold = sklearn.model_selection.KFold(splits, shuffle=True)
    cv_score = sklearn.model_selection.cross_validate(clf, x, y, cv=kfold, scoring={
        'mse': 'neg_mean_squared_error',
        'r2': 'r2'
    }, return_train_score=True)
    return cv_score

In [4]:
def run_cross_validation(x, y, model, params={}):
    cv_result = cv(x, y, model, params=params, splits=5)
    for scorer, score in cv_result.items():
        print('%s: %s' % (scorer, score))
        print('Average %s: %f' % (scorer, score.mean()))

In [5]:
params = {
#         'boosting_type': 'gbdt',
#         "num_leaves": 200,
#         "max_depth": -1,
#         "learning_rate": 0.1,
#         'min_child_samples': 100,
#         'n_estimators': 200,
#         'learning_rate': 0.05,
#         'boost_from_average': True,
#         'min_child_weight': 1e-3,
#         "subsample_for_bin": 20000,
#         'max_bin': 512,
#         'reg_alpha': 3,
#         'reg_lambda': 5,
#         'subsample': 0.9,
#         'colsample_bytree':0.7, 
#         'subsample_freq': 1,
#         'n_jobs': -1,
}

In [6]:
run_cross_validation(trains, labels, lgb.LGBMRegressor, params)

fit_time: [ 30.77630401  27.62138534 161.10954428  25.75708294  25.99972391]
Average fit_time: 54.252808
score_time: [5.21686196 4.5267868  4.49338889 4.8902638  4.75400782]
Average score_time: 4.776262
test_mse: [-0.33999266 -0.33278701 -0.33501415 -0.33678097 -0.33627152]
Average test_mse: -0.336169
train_mse: [-0.33380626 -0.3358412  -0.33500716 -0.33485157 -0.33461279]
Average train_mse: -0.334824
test_r2: [0.15380684 0.15673228 0.15432648 0.15521003 0.15490938]
Average test_r2: 0.154997
train_r2: [0.1588387  0.15750627 0.15880141 0.15786735 0.15886183]
Average train_r2: 0.158375
